# Preparación de datasets

## BDD100K

### Conversión de JSON a TXT

In [2]:
import json
import os
from pathlib import Path
from tqdm import tqdm

In [3]:
CLASS_MAPPING = {
    'car': 0,
    'person': 1,
    'motorcycle': 2,
    'bicycle': 3,
    'truck': 4,
    'bus': 5,
    'rider': 6,
}

In [4]:
def convert_bdd100k_to_yolo(json_dir, output_dir, class_mapping, img_width=1280, img_height=720):

    json_dir = Path(json_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    json_files = list(json_dir.glob('*.json'))
    
    print(f"Encontrados {len(json_files)} archivos JSON")
    print(f"Dimensiones de imagen: {img_width}x{img_height}")
    
    converted = 0
    skipped = 0
    total_objects = 0
    class_counts = {cls: 0 for cls in class_mapping.keys()}
    
    for json_file in tqdm(json_files, desc="Convirtiendo"):
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            yolo_lines = []
            
            # Procesar objetos del primer frame
            if 'frames' in data and len(data['frames']) > 0:
                objects = data['frames'][0].get('objects', [])
            else:
                objects = []
            
            for obj in objects:
                category = obj.get('category', '')
                
                # Filtrar solo las clases que nos interesan
                if category not in class_mapping:
                    continue
                
                class_id = class_mapping[category]
                
                # Extraer bbox
                if 'box2d' in obj:
                    box = obj['box2d']
                    x1 = box['x1']
                    y1 = box['y1']
                    x2 = box['x2']
                    y2 = box['y2']
                    
                    # Convertir a formato YOLO (x_center, y_center, width, height) normalizado
                    x_center = ((x1 + x2) / 2) / img_width
                    y_center = ((y1 + y2) / 2) / img_height
                    width = (x2 - x1) / img_width
                    height = (y2 - y1) / img_height
                    
                    # Validar que los valores estén en rango [0, 1]
                    if 0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 < width <= 1 and 0 < height <= 1:
                        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
                        total_objects += 1
                        class_counts[category] += 1
            
            # Guardar archivo TXT solo si tiene anotaciones
            if yolo_lines:
                # Usar el nombre del campo "name" del JSON
                image_name = data.get('name', json_file.stem)
                txt_file = output_dir / f"{image_name}.txt"
                with open(txt_file, 'w') as f:
                    f.write('\n'.join(yolo_lines))
                converted += 1
            else:
                skipped += 1
                
        except Exception as e:
            print(f"\nError procesando {json_file.name}: {e}")
            skipped += 1
    
    print(f"\n✓ Convertidos: {converted} archivos")
    print(f"✗ Omitidos: {skipped} archivos (sin objetos de interés)")
    print(f"📦 Total objetos: {total_objects}")
    print(f"\n📊 Objetos por clase:")
    for cls, count in class_counts.items():
        if count > 0:
            print(f"  - {cls}: {count}")
    print(f"\n📁 Guardados en: {output_dir}")

#### Train

In [5]:
JSON_DIR = r"C:\Users\adria\Bootcamp\ML_The_Bridge\datasets\bdd100k_labels\100k\train"
OUTPUT_DIR = r"C:\Users\adria\Bootcamp\ML_The_Bridge_datasets\Train\labels"

if __name__ == "__main__":
    convert_bdd100k_to_yolo(JSON_DIR, OUTPUT_DIR, CLASS_MAPPING)

Encontrados 70000 archivos JSON
Dimensiones de imagen: 1280x720


Convirtiendo: 100%|██████████| 70000/70000 [10:52<00:00, 107.30it/s]


✓ Convertidos: 69533 archivos
✗ Omitidos: 467 archivos (sin objetos de interés)
📦 Total objetos: 851530

📊 Objetos por clase:
  - car: 713917
  - person: 91405
  - truck: 30003
  - bus: 11684
  - rider: 4521

📁 Guardados en: C:\Users\adria\Bootcamp\ML_The_Bridge_datasets\Train\labels


#### Test

In [6]:
JSON_DIR = r"C:\Users\adria\Bootcamp\ML_The_Bridge\datasets\bdd100k_labels\100k\test"
OUTPUT_DIR = r"C:\Users\adria\Bootcamp\ML_The_Bridge_datasets\Test\labels"

if __name__ == "__main__":
    convert_bdd100k_to_yolo(JSON_DIR, OUTPUT_DIR, CLASS_MAPPING)

Encontrados 20000 archivos JSON
Dimensiones de imagen: 1280x720


Convirtiendo: 100%|██████████| 20000/20000 [02:13<00:00, 149.67it/s]


✓ Convertidos: 19859 archivos
✗ Omitidos: 141 archivos (sin objetos de interés)
📦 Total objetos: 242946

📊 Objetos por clase:
  - car: 205094
  - person: 24641
  - truck: 8701
  - bus: 3217
  - rider: 1293

📁 Guardados en: C:\Users\adria\Bootcamp\ML_The_Bridge_datasets\Test\labels


#### Validation

In [7]:
JSON_DIR = r"C:\Users\adria\Bootcamp\ML_The_Bridge\datasets\bdd100k_labels\100k\val"
OUTPUT_DIR = r"C:\Users\adria\Bootcamp\ML_The_Bridge_datasets\Val\labels"

if __name__ == "__main__":
    convert_bdd100k_to_yolo(JSON_DIR, OUTPUT_DIR, CLASS_MAPPING)

Encontrados 10000 archivos JSON
Dimensiones de imagen: 1280x720


Convirtiendo: 100%|██████████| 10000/10000 [01:33<00:00, 106.97it/s]


✓ Convertidos: 9921 archivos
✗ Omitidos: 79 archivos (sin objetos de interés)
📦 Total objetos: 122259

📊 Objetos por clase:
  - car: 102506
  - person: 13262
  - truck: 4245
  - bus: 1597
  - rider: 649

📁 Guardados en: C:\Users\adria\Bootcamp\ML_The_Bridge_datasets\Val\labels


# Descarto este notebook

Voy a usar un dataset distinto porque este me da muchos problemas